In [1]:
import stardog
import json

#tbio-conn-admin-ptc.json
adminFile = '/Users/vincent/Projects/TBIO/tbio-conn-admin-local.json'
conn_details = {}
with open(adminFile, 'r') as readFile:
    conn_details = json.loads(readFile.read())

# Download the member of the family

In [2]:
def getMembers(queryRes):
    members = []
    for personVal in queryRes['results']['bindings']:
        person = personVal['personVal']['value']
        if person not in members:
            members.append(person)
#         print(personVal['personVal']['value'])
    return members

In [3]:
def familyMemberQry(familyName):
    return """SELECT DISTINCT ?personVal WHERE {
    GRAPH ?graph {
        ?person a tbio:Person .
        ?person tbio:isMemberOfFamily ?familyEvt .
        ?familyEvt tbio:hasFamily <http://tbio.orient.cas.cz#%s> .
        BIND(STR(?person) AS ?personStr) .
        BIND(REPLACE(?personStr, "http://tbio.orient.cas.cz#", "") AS ?personVal) .
    }
}""" % (familyName)

# 台灣五大族

In [4]:
oldFamilies = [
'林文察家系',
'林文明家系',
'林獻堂家系',
'林維德家系',
'林維讓家系',
'林維源家系',
'基隆顏家家族',
'辜顯榮家系',
'辜振甫家系',
'陳啓清家系',
'陳啓川家系'
]

In [5]:
oldFamilyWMembers = {}
with stardog.Connection('tbio', **conn_details) as conn:
    for family in oldFamilies:
        query = familyMemberQry(family)

        # dic
        results = conn.select(query)
        members = getMembers(results)
        
        # '林文察家系': ['楊天錫', '林培英', '施璇璣', ...], '林文明家系': [...]
        oldFamilyWMembers[family] = members

# oldFamilyWMembers

# 新五家

In [6]:
newFamilies = [
'國泰蔡家家系',
'吳三連家族',
'蔡萬才家系',
'何義家系',
'何傳家系',
'何永家系',
'吳火獅家系',
'侯雨利家系',
'吳修齊家系'
]

In [7]:
newFamilyWMembers = {}
with stardog.Connection('tbio', **conn_details) as conn:
    for family in newFamilies:
        query = familyMemberQry(family)

        # dic
        results = conn.select(query)
        members = getMembers(results)
        
        # '林文察家系': ['楊天錫', '林培英', '施璇璣', ...], '林文明家系': [...]
        newFamilyWMembers[family] = members

# newFamilyWMembers

# member connected?

In [8]:
def relationConnect(queryRes):
    connection = []
    for result in queryRes['results']['bindings']:
        relation = result['p']['value'].replace('http://tbio.orient.cas.cz#', '')
        fromWho = result['x']['value'].replace('http://tbio.orient.cas.cz#', '')
        toWho = result['y']['value'].replace('http://tbio.orient.cas.cz#', '')
        
        if len(connection) == 0:
            connection.append(fromWho)
        if connection[-1] != fromWho:
            connection.append(fromWho)
        connection.append(relation)
        connection.append(toWho)

    return connection

In [9]:
def familyRelationQry(person1, person2):
    return """PATHS 
    START ?x = <http://tbio.orient.cas.cz#%s> 
    END ?y = <http://tbio.orient.cas.cz#%s> 
VIA {
    ?x ?p ?y .
    ?p rdfs:subPropertyOf ?familyOP .
    FILTER ( ?familyOP = tbio:hasFamilyRelation || ?familyOP = tbio:isFamilyRelationOf ) .
} LIMIT 1""" % (person1, person2)

In [10]:
def queryNewFamily(oldfamily, oldperson):
    connList = []
    for newfamily in newFamilyWMembers:
        newmembers = newFamilyWMembers[newfamily]
        for newperson in newmembers:

            query = familyRelationQry(oldperson, newperson)

            # dic
            results = conn.paths(query)
            connection = relationConnect(results)
#             print(oldfamily, oldperson, newfamily, newperson, connection)
            
            if len(connection) > 0:
                connList.append([oldfamily, oldperson, newfamily, newperson, connection])
                break
#             return connection
    return connList

def queryConnection():
    connectList = []
    with stardog.Connection('tbio', **conn_details) as conn:
        for oldfamily in oldFamilyWMembers:
            oldmembers = oldFamilyWMembers[oldfamily]
            for oldperson in oldmembers:
                connList = queryNewFamily(oldfamily, oldperson)
                connectList.extend(connList)
                break

    return connectList

In [11]:
resList = queryConnection()

In [12]:
resList[0:1]

[['林文察家系',
  '楊天錫',
  '國泰蔡家家系',
  '蔡淑貞',
  ['楊天錫',
   'hasBrother',
   '楊肇嘉',
   'hasDaughter',
   '楊湘薰',
   'hasHusband',
   '蔡萬才',
   'hasBrother',
   '蔡萬春',
   'hasDaughter',
   '蔡淑貞']]]

In [13]:
import pandas as pd
finalDf = pd.DataFrame(resList, columns=['SourceFamily', 'SourcePerson', 'TargetFamily', 'TargetPerson', 'Path'])
finalDf.head()

,SourceFamily,SourcePerson,TargetFamily,TargetPerson,Path
0,林文察家系,楊天錫,國泰蔡家家系,蔡淑貞,"[楊天錫, hasBrother, 楊肇嘉, hasDaughter, 楊湘薰, hasHu..."
1,林文察家系,楊天錫,吳三連家族,吳三連,"[楊天錫, hasBrother, 楊天賦, hasDaughter, 楊素娥, hasHu..."
2,林文察家系,楊天錫,蔡萬才家系,楊湘薰,"[楊天錫, hasBrother, 楊肇嘉, hasDaughter, 楊湘薰]"
3,林文察家系,楊天錫,何義家系,何傳,"[楊天錫, hasBrother, 楊天賦, hasSon, 楊基炘, hasWife, 何..."
4,林文察家系,楊天錫,何傳家系,何傳,"[楊天錫, hasBrother, 楊天賦, hasSon, 楊基炘, hasWife, 何..."


In [14]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/20190523'
write_file_to = '{0}/{1}'.format(filepath, 'familyToFamily_20190523_v1.xlsx')
finalDf.to_excel(write_file_to, index=False)

# Another way to get connections

In [15]:
# import pandas as pd

In [16]:
# filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/20190523'

# filename = 'familyRelations_20190516.tsv'

# read_filename = '{0}/{1}'.format(filepath, filename)

In [17]:
# workDf = pd.read_csv(read_filename, delimiter='\t')
# workDf = workDf.fillna('')
# workDf.shape, workDf.head()

In [18]:
# workList = []
# for idx in range(0, len(workDf)):
#     row = workDf.loc[idx]

#     sPerson = str(row['?personVal'])
#     relation = str(row['?relationVal'])
#     tPerson = str(row['?toPVal'])
    
#     if sPerson == '' or tPerson == '':
#         continue
        
#     toRow = [sPerson, relation, tPerson]

#     if toRow not in workList:
#         workList.append(toRow)
#     else:
#         print("DUP!", toRow)
    
# print(len(workList))

# Filter out inverse relations

In [19]:
# def findInverse(INsPerson, INtPerson):
#     for work in workList:
#         sPerson = work[0]
#         relation = work[1]
#         tPerson = work[2]
        
# #         if INsPerson == sPerson and INtPerson == tPerson:
# # #             print(INsPerson, INtPerson, work)
# #             return relation
#         if INsPerson == tPerson and INtPerson == sPerson:
# #             print(INsPerson, INtPerson, work)
#             return relation
#     return ''

# relationList = []
# for work in workList:
#     sPerson = work[0]
#     relation = work[1]
#     tPerson = work[2]

#     if 'has' in relation:
#         relationList.append(work)
#         continue
    
#     OUTrelation = findInverse(sPerson, tPerson)
#     if OUTrelation == '':
#         relationList.append(work)
        
# #     if 'has' in OUTrelation:
# #         resList.append([sPerson, OUTrelation, tPerson])
# #     elif OUTrelation != '':
# #         resList.append(work)

# print(len(relationList), relationList[0:5])

# Run connection finding

In [20]:
# def findRelation(INsPerson, INtPerson):
#     for triple in relationList:
#         sPerson = triple[0]
#         relation = triple[1]
#         tPerson = triple[2]
        
#         if INsPerson == sPerson and INtPerson == tPerson:
#             return [INsPerson, relation, INtPerson]
        
#         if INsPerson == tPerson and INtPerson == tPerson:
#             return [INsPerson, "-{}".format(relation), INtPerson]
#     return None

In [21]:
# def runConnection():
#     count = 0
#     for oldfamily in oldFamilyWMembers:
#         oldmembers = oldFamilyWMembers[oldfamily]
#         for oldperson in oldmembers:

#             for newfamily in newFamilyWMembers:
#                 newmembers = newFamilyWMembers[newfamily]
#                 for newperson in newmembers:
#                     print(oldperson, newperson)
#                     print(findRelation(oldperson, newperson))
#                     if count >= 3:
#                         return
#                     count += 1

In [22]:
# runConnection()